In [ ]:
import glob
import json
import urllib2
import datetime
import time

email = "bgreshake@googlemail.com"

In [ ]:

pubmed_files = glob.glob("../data/*pubmed-crawl.csv")
latest_pubmed = pubmed_files[-1]
print("reading file %s" % latest_pubmed)
dois = []
with open(latest_pubmed) as pubmed_file:
    for line in pubmed_file:
        if line[0] != "#":
            la = line.strip().split("\t")
            if la[-1] != "DOI":
                dois.append(la[-1])

In [ ]:
# get OA information from oaDOI.org API: 

pubmed_files = glob.glob("../data/*pubmed-crawl.csv")
latest_pubmed = pubmed_files[-1]
print("reading file %s" % latest_pubmed)
dois = []
with open(latest_pubmed) as pubmed_file:
    for line in pubmed_file:
        if line[0] != "#":
            la = line.strip().split("\t")
            if la[-1] != "DOI":
                dois.append(la[-1])


oadoi_output = []
crawltime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

for i,doi in enumerate(dois):
    request = "https://api.oadoi.org/v2/%s?email=%s" % (doi,email)
    error_counter = 0
    if i % 100 == 0:
        print "finished %s DOIs" % str(i)
    while error_counter < 5:
        try:
            data = json.load(urllib2.urlopen(request))
            oadoi_output.append(data)
            break
        except:
            time.sleep(2)
            error_counter += 1
            print("error %s for %s" % (str(error_counter),doi))
    else:
        print("ERROR: %s failed terminally" % doi)


In [ ]:
# ok, let's save the output we got from the oaDOI API
# let's save the full json we got as well as the key things as CSV

# "../data/parsetime-oadoi.csv" contains all DOI plus key OA-metrics
# "../data/parsetime-oadoi.json" contains all json delivered by oaDOI
# the CSV file has a header with the oaDOI field names

outfile = "../data/%s-oadoi.csv" % crawltime

output = open(outfile,"w")
output.write("#infile: %s \n" % latest_pubmed)
output.write("doi\tis_oa\tjournal_is_oa\thost_type\n")

outfile_json = "../data/%s-oadoi.json" % crawltime
output_json = open(outfile_json,"w")
json.dump(oadoi_output, output_json)
output_json.close()

for paper in oadoi_output:
    doi = paper["doi"]
    is_oa = paper["is_oa"]
    journal_is_oa = paper["journal_is_oa"]
    if is_oa == True:
        host_type = paper["best_oa_location"]["host_type"]
    else:
        host_type = "NA"
    output.write("%s\t%s\t%s\t%s\n" % (doi,str(is_oa),str(journal_is_oa),host_type))